In [1]:
!pip install pyspark

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
spark

In [3]:
from pyspark.ml.linalg import Vectors

denseVec = Vectors.dense(1.0, 2.0, 3.0)
size = 3
idx = [1, 2] # locations of non-zero elements in vector
values = [2.0, 3.0]
sparseVec = Vectors.sparse(size, idx, values)

sparseVec

SparseVector(3, {1: 2.0, 2: 3.0})

In [4]:
censu = spark   \
    .read   \
    .option("inferSchema", "true")  \
    .option("header", "true")   \
    .csv("./data/census.csv")

censu.show()

+---+-----------------+------------+-------------+-------------+--------------------+------------------+--------------+-------------------+-------+------------+------------+-------------+--------------+------+
|age|        workclass|final-weight|    education|education-num|      marital-status|        occupation|  relationship|               race|    sex|capital-gain|capital-loos|hour-per-week|native-country|income|
+---+-----------------+------------+-------------+-------------+--------------------+------------------+--------------+-------------------+-------+------------+------------+-------------+--------------+------+
| 39|        State-gov|     77516.0|    Bachelors|         13.0|       Never-married|      Adm-clerical| Not-in-family|              White|   Male|      2174.0|         0.0|         40.0| United-States| <=50K|
| 50| Self-emp-not-inc|     83311.0|    Bachelors|         13.0|  Married-civ-spouse|   Exec-managerial|       Husband|              White|   Male|         0.0|

In [5]:
from pyspark.ml.feature import RFormula
supervised = RFormula(formula="income ~ . + age + workclass + education + occupation + race + sex")

In [6]:
fittedRF = supervised.fit(censu)
preparedDF = fittedRF.transform(censu)
preparedDF.show()

+---+-----------------+------------+-------------+-------------+--------------------+------------------+--------------+-------------------+-------+------------+------------+-------------+--------------+------+--------------------+-----+
|age|        workclass|final-weight|    education|education-num|      marital-status|        occupation|  relationship|               race|    sex|capital-gain|capital-loos|hour-per-week|native-country|income|            features|label|
+---+-----------------+------------+-------------+-------------+--------------------+------------------+--------------+-------------------+-------+------------+------------+-------------+--------------+------+--------------------+-----+
| 39|        State-gov|     77516.0|    Bachelors|         13.0|       Never-married|      Adm-clerical| Not-in-family|              White|   Male|      2174.0|         0.0|         40.0| United-States| <=50K|(100,[0,5,9,12,25...|  0.0|
| 50| Self-emp-not-inc|     83311.0|    Bachelors|  

In [7]:
train, test = preparedDF.randomSplit([0.7, 0.3])

In [8]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(labelCol="label",featuresCol="features")

In [9]:
print(lr.explainParams())

aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0)
family: The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial (default: auto)
featuresCol: features column name. (default: features, current: features)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label, current: label)
lowerBoundsOnCoefficients: The lower bounds on coefficients if fitting under bound constrained optimization. The bound matrix must be compatible with the shape (1, number of features) for binomial regression, or (number of classes, number of features) for multinomial regression. (undefined)
lowerBoundsOnIntercepts: The lower bounds on intercepts if fitting under bound constrained optimization. The

In [10]:
fittedLR = lr.fit(train)

In [11]:
fittedLR.transform(train).select("label", "prediction").show()

+-----+----------+
|label|prediction|
+-----+----------+
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
+-----+----------+
only showing top 20 rows



## Pipelining Our Workflow

In [12]:
train, test = censu.randomSplit([0.7, 0.3])

In [13]:
rForm = RFormula()
lr = LogisticRegression().setLabelCol("label").setFeaturesCol("features")

In [14]:
from pyspark.ml import Pipeline

stages = [rForm, lr]
pipeline = Pipeline().setStages(stages)

## Training and Evaluation

In [15]:
from pyspark.ml.tuning import ParamGridBuilder

params = ParamGridBuilder() \
    .addGrid(rForm.formula, [
        "income ~ . + age + workclass + education + occupation + race + sex"])  \
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])   \
    .addGrid(lr.regParam, [0.1, 2.0])   \
    .build()

In [16]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator() \
    .setMetricName("areaUnderROC")  \
    .setRawPredictionCol("prediction")  \
    .setLabelCol("label")

In [17]:
from pyspark.ml.tuning import TrainValidationSplit

tvs = TrainValidationSplit()    \
    .setTrainRatio(0.75)    \
    .setEstimatorParamMaps(params)  \
    .setEstimator(pipeline) \
    .setEvaluator(evaluator)

In [18]:
tvsFitted = tvs.fit(train)

In [19]:
evaluator.evaluate(tvsFitted.transform(test))

0.7112305816576201